In [2]:
import urllib2
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from urlparse import urlsplit, urljoin

In [3]:
base_url = 'http://www.cra-arc.gc.ca/formspubs/t1gnrl/llyrs-eng.html'
base_page = urllib2.urlopen(base_url).read()
selection = Selector(text=base_page)

### Get a url for each year

In [4]:
year_pages = selection.xpath("//main[@property='mainContentOfPage']/ul/li/a/@href").extract()
year_pages
year_pages_dict = {}
year = 2014
for year_page in year_pages:
    year_pages_dict[str(year)] = year_page
    year = year - 1

In [6]:
url_split = urlsplit(base_url)
year_pages_dict_full = {}
for year, url in year_pages_dict.iteritems():
    # Create the url and then append it into the empty array
    url_joined = 'http://' + url_split.netloc + url
    year_pages_dict_full[year] = url_joined

# Debugging
year_pages_dict_full

{'1985': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1985/menu-e.html',
 '1986': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/menu-e.html',
 '1987': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1987/menu-e.html',
 '1988': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1988/menu-e.html',
 '1989': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1989/menu-e.html',
 '1990': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1990/menu-e.html',
 '1991': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1991/menu-e.html',
 '1992': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1992/menu-e.html',
 '1993': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1993/menu-e.html',
 '1994': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1994/menu-e.html',
 '1995': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1995/menu-e.html',
 '1996': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1996/menu-e.html',
 '1997': u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1997/

In [8]:
provincial_links = {}

def provinceUrlListAppend(url_list, flag, year_url, year):
    print year
    if flag == "normal" or flag == "older":
        for url in url_list:
            provincial_links[year].append('http://' + url_split.netloc + url)
    elif flag == "reallyOld":
        for url in url_list:
            link = 'http://' + url_split.netloc
            link += urlsplit(year_url).path.rsplit('/', 1)[0]
            link += "/" + url
            provincial_links[year].append(link)

def provinceUrlListProcess(url_list, year_url, year):
    provincial_links[year] = []
    # If it's 'normal' and Alberta is detected, go ahead
    if url_list[0] == 'Alberta' or url_list[0] == ' ':
        url_list = selection.xpath("//main[@property='mainContentOfPage']/ul[1]/li/a/@href").extract()
        provinceUrlListAppend(url_list, "normal", year_url, year)
    else:
        # In some older links, they stuck another unordered list above
        url_list = selection.xpath("//main[@property='mainContentOfPage']/ul[2]/li/a/@href").extract()
        # Another edge case check where they started using relative urls around 1998
        if url_list[0][0] == "/":
            provinceUrlListAppend(url_list, "older", year_url, year)
        else:
            provinceUrlListAppend(url_list, "reallyOld", year_url, year)

for year, year_url in year_pages_dict_full.iteritems():
    year_page = urllib2.urlopen(year_url).read()
    selection = Selector(text=year_page)
    provt_pages = selection.xpath("//main[@property='mainContentOfPage']/ul/li/a/text()").extract()
    provinceUrlListProcess(provt_pages, year_url, year)

print provincial_links

1986
1987
1985
1988
1989
2014
2011
2010
2013
2012
1991
1990
1993
1992
1995
1994
1997
1996
1999
1998
2002
2003
2000
2001
2006
2007
2004
2005
2008
2009
{'1986': [u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/alberta-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/british-columbia-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/manitoba-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/new-brunswick-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/newfoundland-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/northwest-territories-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/nova-scotia-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/ontario-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/prince-edward-island-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/quebec-e.html', u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1986/saskatchewan-e

In [10]:
year = provincial_links["2014"]
url = year[0]
print url
page = urllib2.urlopen(url).read()
selection = Selector(text=base_page)
selection

http://www.cra-arc.gc.ca/formspubs/t1gnrl/ab-eng.html


<Selector xpath=None data=u'<html class="no-js" lang="en">\r\n<!--<![e'>